In [167]:
import pandas as pd
from mhcflurry import Class1AffinityPredictor
import os
import mhcnames
import rank_model
import numpy as np

In [101]:
def normalize(s, disallowed=["MIC", "HFE"]):
    if any(item in s for item in disallowed):
        return None
    try:
        return mhcnames.normalize_allele_name(s)
    except:
        while s:
            s = ":".join(s.split(":")[:-1])
            try:
                return mhcnames.normalize_allele_name(s)
            except:
                pass
        return None

In [20]:
allele_names = pd.read_csv("C:/Users/paul_/OneDrive/Desktop/master-thesis/data/new_model_testset.csv").allele.unique()
predictor = Class1AffinityPredictor.load()

mhc_names = list(set(allele_names.tolist()) & set(predictor.supported_alleles))


In [49]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [51]:
data = pd.read_csv("C:/Users/paul_/OneDrive/Desktop/master-thesis/data/new_model_testset.csv")
mask = (data['peptide'].str.len() >= 8) & (data['peptide'].str.len() <= 15)
test_data = data.loc[mask]
test_data[test_data.allele.isin(mhc_names)]

d = { "*": "_", ":": "_"}


for i in mhc_names:
    
    df = test_data[test_data.allele == i]
    
    name = replace_all(i, d)
    df.peptide.to_csv("C:/Users/paul_/OneDrive/Desktop/master-thesis/data/netmhc/netmhcpan_" + name + ".csv")

In [89]:
directory = "C:/Users/paul_/OneDrive/Desktop/master-thesis/data/netmhc"
list_df = []

for filename in os.listdir(directory):
    if filename.endswith(".xls"): 
        head = pd.read_csv(os.path.join(directory, filename), sep='\t', lineterminator='\r')
        mhc_typ = list(head.columns)[3]
        data = pd.read_csv(os.path.join(directory, filename), sep='\t', lineterminator='\r', skiprows =1)
        data = data.loc[:, ["Peptide", 'nM']]
        data['mhc'] = list(head.columns)[3]
        data['mhcpan_version'] = filename[6]
        list_df.append(data)
        continue
    else:
        continue

In [131]:
mhcpan_data = pd.concat(list_df)
mhcpan_data.dropna().reset_index(drop=True)
mhcpan_data['allele'] = mhcpan_data['mhc'].apply(normalize)
mhcpan_data['prediction'] = mhcpan_data['nM']
mhcpan_data = mhcpan_data.loc[:, ['Peptide', 'prediction', 'mhcpan_version', 'allele']]
mhcpan_data = mhcpan_data.drop_duplicates().reset_index(drop = True)

In [143]:
complete_data = pd.merge(mhcpan_data,
                         test_data.loc[:, ['peptide', 'allele', 'measurement_value']],
                         left_on = ["Peptide", 'allele'],
                         right_on = ["peptide", 'allele'],
                         how='inner')
complete_data.rename(columns={'measurement_value':'true',
                          'prediction':'pred'}, 
                     inplace=True)

In [150]:
allele_df = complete_data[complete_data['allele'] == str('H-2-Db')]
allele_df[allele_df['mhcpan_version'] == str(3)].head()

,Peptide,pred,mhcpan_version,allele,peptide,true
0,AAIRGNDVI,74.1739,3,H-2-Db,AAIRGNDVI,10.7505
1,AAIRGNDVI,74.1739,3,H-2-Db,AAIRGNDVI,10.8000
4,ALPTNEFISFL,4027.1538,3,H-2-Db,ALPTNEFISFL,79.2139
5,ALPTNEFISFL,4027.1538,3,H-2-Db,ALPTNEFISFL,79.2000
8,APGSPTNLEFI,370.4219,3,H-2-Db,APGSPTNLEFI,14.0395


In [172]:
mhc_names = list(set(allele_names.tolist()) & set(predictor.supported_alleles))
combined_dict = {}
len_dict = {}

for i in mhc_names:
    if i == "Mamu-A*01:01":
        print(i)
    else:
        loop_df = complete_data[complete_data['allele'] == str(i)]
        error_dict_netmhc3, len_dict_netmhc3 = rank_model.root_mean_squared(loop_df[loop_df['mhcpan_version'] == str(3)], str(i), "netmhc3")
        error_dict_netmhc4, len_dict_netmhc4 = rank_model.root_mean_squared(loop_df[loop_df['mhcpan_version'] == str(4)], str(i), "netmhc4")
        len_dict.update(len_dict_netmhc3)
        combined_dict.update(error_dict_netmhc3)
        len_dict.update(len_dict_netmhc4)
        combined_dict.update(error_dict_netmhc4)

Mamu-A*01:01


In [173]:
combined_dict

{'HLA-C*06:02netmhc3': 673.46,
 'HLA-C*06:02netmhc4': 664.7986,
 'HLA-A*03:01netmhc3': 183071.60158227387,
 'HLA-A*03:01netmhc4': 183046.29313147717,
 'H-2-Kbnetmhc3': 8344.708687567972,
 'H-2-Kbnetmhc4': 8386.806046103065,
 'HLA-A*11:01netmhc3': 4896.057780953406,
 'HLA-A*11:01netmhc4': 5019.252430159365,
 'HLA-A*68:01netmhc3': 7.103235879864895,
 'HLA-A*68:01netmhc4': 13.77033802998314,
 'HLA-B*58:01netmhc3': 1971.647715537852,
 'HLA-B*58:01netmhc4': 2437.8959209294267,
 'HLA-C*03:03netmhc3': 595.6867,
 'HLA-C*03:03netmhc4': 538.6556,
 'HLA-B*07:02netmhc3': 290702.3364150556,
 'HLA-B*07:02netmhc4': 290660.16511980735,
 'HLA-A*24:02netmhc3': 15185.153204932887,
 'HLA-A*24:02netmhc4': 15086.564850557073,
 'HLA-B*15:01netmhc3': 18773.430708507767,
 'HLA-B*15:01netmhc4': 19176.395690068963,
 'HLA-B*53:01netmhc3': 1677.7648945520239,
 'HLA-B*53:01netmhc4': 1756.6716614282075,
 'HLA-B*57:01netmhc3': 308.2082071913914,
 'HLA-B*57:01netmhc4': 711.4158424107128,
 'HLA-A*02:05netmhc3': 14725.6

In [174]:
len_dict

{'HLA-C*06:02netmhc3': 1,
 'HLA-C*06:02netmhc4': 1,
 'HLA-A*03:01netmhc3': 30,
 'HLA-A*03:01netmhc4': 30,
 'H-2-Kbnetmhc3': 106,
 'H-2-Kbnetmhc4': 106,
 'HLA-A*11:01netmhc3': 19,
 'HLA-A*11:01netmhc4': 19,
 'HLA-A*68:01netmhc3': 4,
 'HLA-A*68:01netmhc4': 4,
 'HLA-B*58:01netmhc3': 3,
 'HLA-B*58:01netmhc4': 3,
 'HLA-C*03:03netmhc3': 1,
 'HLA-C*03:03netmhc4': 1,
 'HLA-B*07:02netmhc3': 47,
 'HLA-B*07:02netmhc4': 47,
 'HLA-A*24:02netmhc3': 27,
 'HLA-A*24:02netmhc4': 27,
 'HLA-B*15:01netmhc3': 10,
 'HLA-B*15:01netmhc4': 10,
 'HLA-B*53:01netmhc3': 12,
 'HLA-B*53:01netmhc4': 12,
 'HLA-B*57:01netmhc3': 12,
 'HLA-B*57:01netmhc4': 12,
 'HLA-A*02:05netmhc3': 16,
 'HLA-A*02:05netmhc4': 16,
 'HLA-A*01:01netmhc3': 33,
 'HLA-A*01:01netmhc4': 33,
 'HLA-A*02:01netmhc3': 155,
 'HLA-A*02:01netmhc4': 155,
 'HLA-B*08:01netmhc3': 4,
 'HLA-B*08:01netmhc4': 4,
 'HLA-A*68:02netmhc3': 4,
 'HLA-A*68:02netmhc4': 4,
 'HLA-B*38:01netmhc3': 12,
 'HLA-B*38:01netmhc4': 12,
 'HLA-B*44:03netmhc3': 14,
 'HLA-B*44:03netmhc

In [175]:
weighted_a = np.average(a = np.array([value for key, value in combined_dict.items() if 'netmhc3' in key.lower()]),
                        weights = np.array([value for key, value in len_dict.items() if 'netmhc3' in key.lower()]))
a = np.mean(a = np.array([value for key, value in combined_dict.items() if 'netmhc3' in key.lower()]))
print(weighted_a)
print(a)

72090.9502087772
59119.46956194531


In [176]:
weighted_b = np.average(a = np.array([value for key, value in combined_dict.items() if 'netmhc4' in key.lower()]),
                        weights = np.array([value for key, value in len_dict.items() if 'netmhc4' in key.lower()]))
b = np.mean([value for key, value in combined_dict.items() if 'netmhc4' in key.lower()])
print(weighted_b)
print(b)

72127.56765750614
59272.67712745496
